In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')
s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp
import excel_formating

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase

In [ ]:
#import data with the users mails
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Coverage%20Advisor%20Tracker%20OFICIAL.xlsx"\
    ,filename = 'Coverage Advisor Tracker OFICIAL.xlsx')
ca = pd.read_excel ('Coverage Advisor Tracker OFICIAL.xlsx')
ca.dropna(axis=0, subset=['Account'], inplace=True)
ca.drop(list(ca[ca['Account']==0].index), axis=0, inplace=True)

In [ ]:
m = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/coverage_mails.xlsx"\
    ,filename = 'coverage_mails.xlsx')
mail = pd.read_excel('coverage_mails.xlsx')

In [ ]:
#SMTP Mail variables
today = datetime.strftime(datetime.today(), '%m-%d-%Y')
myMail = 'testNub78@outlook.com'
password = 'EasyP4ssword78'
subjectEmail = 'Coverage error report'

msg1 = """<h1>Hola</h1>
        <h3>Por favor revisa el archivo adjunto para encontrar el detalle de los errores</h3>
        <h3>Asegurate de que tus registros esten corregidos y al d&iacute;a</h3>"""

In [ ]:


def format_date_column(serie):
    format_date = []
    for index, value in serie.iteritems():
        if type(value) != datetime and type(value) != Timestamp:  
            if type(value) == float or value != value:
                value = datetime.strptime('2030-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            else: 
                value = datetime.strptime('2050-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
        else:
            value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
            format_date.append(value)
    
    return format_date
    


In [ ]:
# Request Type

request_type = ca[['Request ID', 'Request Type']].copy()

for i, row in request_type.iterrows():
    if pd.isnull(row['Request Type']) == True:
        request_type['Request Type'].loc[i] = 'Request Type Empty'
    else:
        request_type['Request Type'].loc[i] = None

request_type.dropna(axis=0, subset=['Request Type'], inplace=True)


In [ ]:
# Join Call

join_call = ca[['Request ID', 'Join Call']].copy()

for i, row in join_call.iterrows():
    if pd.isnull(row['Join Call']) == True:
        join_call['Join Call'].loc[i] = 'Join Call Empty'
    else:
        join_call['Join Call'].loc[i] = None

join_call.dropna(axis=0, subset=['Join Call'], inplace=True)


In [ ]:
# Upcoming Renewal
upcoming_renewal = ca[['Request ID', 'Upcoming Renewal']].copy()

for index, row in upcoming_renewal.iterrows():
    if pd.isna(row['Upcoming Renewal']) == True:
        upcoming_renewal['Upcoming Renewal'].loc[index] = 'Empty Value'
    elif len(row['Upcoming Renewal']) > 6:v
        upcoming_renewal['Upcoming Renewal'].loc[index] = 'Value with space'
    else:    
        upcoming_renewal['Upcoming Renewal'].loc[index] = None

upcoming_renewal.dropna(axis=0, subset=['Upcoming Renewal'], inplace=True)


In [ ]:
ca = pd.read_excel ('Coverage Advisor Tracker OFICIAL.xlsx')
ca.dropna(axis=0, subset=['Account'], inplace=True)
ca.drop(list(ca[ca['Account']==0].index), axis=0, inplace=True)

In [170]:
# Date Received

date_received = ca[['Request ID', 'Date received']].copy()
date_received['Date received'] = format_date_column(date_received['Date received'])

In [ ]:
for index, row in date_received.iterrows():
    

In [171]:
date_received

,Request ID,Date received
0,103120.0,2021-10-14
1,102639.0,2021-10-14
2,102732.0,2021-10-14
3,103048.0,2021-10-14
4,103106.0,2021-10-14
...,...,...
1564,115650.0,2030-02-02
1565,116119.0,2030-02-02
1566,111891.0,2030-02-02
1567,115641.0,2030-02-02


### Celdas vacías

In [ ]:
#Recorrido de celdas buscando valores vacios en columnas G H I J  AF/AG/AH AM AN AO
Empty  = ca[
    (ca['Request Type'].isna() == True) | (ca['Join Call'].isna() == True)
    | (ca['Sav ID'].isna() == True) | (ca['Upcoming Renewal'].isna() == True) | (ca['CA Name'].isna() == True) | (ca['Expired Opportunity'].isna() == True)
    | (ca['LDoS Opportunity'].isna() == True) | (ca['Next steps'].isna() == True) | (ca['Link PAR IB Rep'].isna() == True)]

Empty_Errors = Empty.copy()

In [ ]:
Empty_Errors = Empty_Errors[['Request ID','Request Type','Join Call','Sav ID', 'Upcoming Renewal','CA Name', 'Expired Opportunity','LDoS Opportunity', 'Next steps','Link PAR IB Rep']]
Empty_Errors = Empty_Errors.fillna('Empty')
Empty_Errors = Empty_Errors.reset_index(drop=True)

In [ ]:
for index, row in Empty_Errors.iterrows():
    for loc, sheet in row.items():
        if sheet != 'Empty':
            if loc != 'Request ID':
                Empty_Errors[loc].loc[index] = None

In [ ]:
Empty_Errors.rename(columns={'Request Type': 'Empty Request Type', 'Join Call': 'Empty Join Call',
                    'Sav ID': 'Empty Sav ID', 'Upcoming Renewal': 'Empty Upcoming Renewal',
                    'CA Name': 'Empty CA Name', 'Expired Opportunity': 'Empty Expired Opportunity',
                    'LDoS Opportunity': 'Empty LDoS Opportunity', 'Next steps':'Empty Next steps', 'Link PAR IB Rep':'Empty Link PAR IB Rep'}, inplace=True)

### Compare column R-W

In [ ]:
ca2 = ca[['Request ID','Coverage Advisor','PAR Requested','Date First Contact','PAR Received','Date Analysis','Date Invite Sent','Date Scheduled']]

In [ ]:
equality_relation = ca2.copy()

In [ ]:
equality_relation['PAR Requested'] = format_date_column(equality_relation['PAR Requested'])
equality_relation['Date First Contact'] = format_date_column(equality_relation['Date First Contact'])
equality_relation['PAR Received'] = format_date_column(equality_relation['PAR Received'])
equality_relation['Date Analysis'] = format_date_column(equality_relation['Date Analysis'])
equality_relation['Date Invite Sent'] = format_date_column(equality_relation['Date Analysis'])
equality_relation['Date Scheduled'] = format_date_column(equality_relation['Date Scheduled'])

In [ ]:
equality_relation.insert(equality_relation.shape[1], 'PAR Requested_greater_Date First Contact', None)
equality_relation.insert(equality_relation.shape[1], 'Date First Contact_greater_PAR Received', None)
equality_relation.insert(equality_relation.shape[1], 'PAR Received_greater_Date Analysis', None)
equality_relation.insert(equality_relation.shape[1], 'Date Analysis_greater_Date Invite Sent', None)
equality_relation.insert(equality_relation.shape[1], 'Date Invite Sent_greater_Date Scheduled', None)

In [ ]:
for i, value in equality_relation.iterrows():
    if value['PAR Requested'] > value['Date First Contact']:
        if value['PAR Requested'] == Timestamp('2030-2-2 00:00:00') or value['PAR Requested'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['PAR Requested_greater_Date First Contact'].loc[i] = None
        else:
            equality_relation['PAR Requested_greater_Date First Contact'].loc[i] = 'X'

    if value['Date First Contact'] > value['PAR Received']:
        if value['Date First Contact'] == Timestamp('2030-2-2 00:00:00') or value['Date First Contact'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['Date First Contact_greater_PAR Received'].loc[i] = None
        else:
            equality_relation['Date First Contact_greater_PAR Received'].loc[i] = 'X'  

    if value['PAR Received'] > value['Date Analysis']:
        if value['PAR Received'] == Timestamp('2030-2-2 00:00:00') or value['PAR Received'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['PAR Received_greater_Date Analysis'].loc[i] = None  
        else:
            equality_relation['PAR Received_greater_Date Analysis'].loc[i] = 'X'

    if value['Date Analysis'] > value['Date Invite Sent']:
        if value['Date Analysis'] == Timestamp('2030-2-2 00:00:00') or value['Date Analysis'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['Date Analysis_greater_Date Invite Sent'].loc[i] = None
        else:
            equality_relation['Date Analysis_greater_Date Invite Sent'].loc[i] = 'X'        

    if value['Date Invite Sent'] > value['Date Scheduled']:
        if value['Date Invite Sent'] == Timestamp('2030-2-2 00:00:00') or value['Date Invite Sent'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['Date Invite Sent_greater_Date Scheduled'].loc[i] = None
        else: 
            equality_relation['Date Invite Sent_greater_Date Scheduled'].loc[i] = 'X' 

In [ ]:
for i, val in equality_relation.iterrows():

    if val['PAR Requested'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['PAR Requested'].loc[i] = 'Format Error'
    else:
        equality_relation['PAR Requested'].loc[i] = None

    if val['Date First Contact'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date First Contact'].loc[i] = 'Format Error'
    else:
        equality_relation['Date First Contact'].loc[i] = None
    
    if val['PAR Received'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['PAR Received'].loc[i] = 'Format Error'
    else:
        equality_relation['PAR Received'].loc[i] = None

    if val['Date Analysis'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Analysis'].loc[i] = 'Format Error'
    else:
        equality_relation['Date Analysis'].loc[i] = None
    
    if val['Date Invite Sent'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Invite Sent'].loc[i] = 'Format Error'
    else:
        equality_relation['Date Invite Sent'].loc[i] = None
    
    if val['Date Scheduled'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Scheduled'].loc[i] = 'Format Error'
    else:
        equality_relation['Date Scheduled'].loc[i] = None



### Familiar Relation

In [ ]:

ca3 = ca[['Request ID','Account Stage','Status','Reason Declined','Pre-Readout Effort (min)','Readout Duration (min)','Follow Up Effort (min)']]

In [ ]:
familiar_relation = ca3.copy()

In [ ]:
familiar_relation.insert(familiar_relation.shape[1], 'Error relation Account Stage, Status columns', None)
familiar_relation.insert(familiar_relation.shape[1], 'Error Reason Declined', None)
familiar_relation.insert(familiar_relation.shape[1], 'Pre-ReadOut Empty', None)
familiar_relation.insert(familiar_relation.shape[1], 'Empty values in Readout Duration and Follow Up Effort', None)

In [ ]:
for index, status in familiar_relation.iterrows():
    if pd.isnull(status['Account Stage']) == False and pd.isnull(status['Status']) == False:
        if status['Status'][0] != status['Account Stage'][0]:
            familiar_relation['Error relation Account Stage, Status columns'].loc[index] = 'Error'
        else:
            if status['Account Stage'][0] == 'E' and status['Status'][2]== 'D' and pd.isnull(status['Reason Declined']) == True:
                familiar_relation['Error Reason Declined'].loc[index] = 'Error'
        
    if type(status['Pre-Readout Effort (min)']) != int:
        if pd.isnull(status['Pre-Readout Effort (min)']) == True:
            if pd.isnull(status['Status']) == False:
                if status['Status'][2] != 'U':
                    familiar_relation['Pre-ReadOut Empty'].loc[index] = 'Error'

    if pd.isnull(status['Account Stage']) == False:
        if status['Account Stage'][0] == 'F' and (pd.isnull(status['Readout Duration (min)']) == True or pd.isnull(status['Follow Up Effort (min)']) == True):
            familiar_relation['Empty values in Readout Duration and Follow Up Effort'].loc[index] = 'Error'   

In [ ]:
familiar_relation.drop(columns=familiar_relation.columns[1:7], inplace=True)

### Compare column AI-AL


In [ ]:
cov = ca[['Request ID','Follow Up 1','Follow Up 2','Follow Up 3','Closed Date']]

In [ ]:
equality_relation2 = cov.copy()

In [ ]:
equality_relation2['Follow Up 1'] = format_date_column(equality_relation2['Follow Up 1'])
equality_relation2['Follow Up 2'] = format_date_column(equality_relation2['Follow Up 2'])
equality_relation2['Follow Up 3'] = format_date_column(equality_relation2['Follow Up 3'])
equality_relation2['Closed Date'] = format_date_column(equality_relation2['Closed Date'])

In [ ]:
equality_relation2['Follow Up 1_greater_Follow Up 2'] = None
equality_relation2['Follow Up 2_greater_Follow Up 3'] = None
equality_relation2['Follow Up 3_greater_Closed Date'] = None

In [ ]:
for i, value in equality_relation2.iterrows():
    if value['Follow Up 1'] > value['Follow Up 2']:
        if value['Follow Up 1'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 1'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation2['Follow Up 1_greater_Follow Up 2'].loc[i] = None
        else:
            equality_relation2['Follow Up 1_greater_Follow Up 2'].loc[i] = 'X'

    if value['Follow Up 2'] > value['Follow Up 3']:
        if value['Follow Up 2'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 2'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation2['Follow Up 2_greater_Follow Up 3'].loc[i] = None
        else:
            equality_relation2['Follow Up 2_greater_Follow Up 3'].loc[i] = 'X'  

    if value['Follow Up 3'] > value['Closed Date']:
        if value['Follow Up 3'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 3'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation2['Follow Up 3_greater_Closed Date'].loc[i] = None  
        else:
            equality_relation2['Follow Up 3_greater_Closed Date'].loc[i] = 'X'

In [ ]:
for i, val in equality_relation2.iterrows():

    if val['Follow Up 1'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation2['Follow Up 1'].loc[i] = 'Format Error'
    else:
        equality_relation2['Follow Up 1'].loc[i] = None

    if val['Follow Up 2'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation2['Follow Up 2'].loc[i] = 'Format Error'
    else:
        equality_relation2['Follow Up 2'].loc[i] = None
    
    if val['Follow Up 3'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation2['Follow Up 3'].loc[i] = 'Format Error'
    else:
        equality_relation2['Follow Up 3'].loc[i] = None

    if val['Closed Date'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation2['Closed Date'].loc[i] = 'Format Error'
    else:
        equality_relation2['Closed Date'].loc[i] = None
    

### Merging tables

In [ ]:
new_table = equality_relation.merge(familiar_relation, how='left', left_on='Request ID', right_on='Request ID').merge(    
            Empty_Errors, how='left', left_on='Request ID', right_on='Request ID').merge(
            equality_relation2, how='left', left_on='Request ID', right_on='Request ID')
new_table.dropna(axis=0, how='all', subset=new_table.columns[2:], inplace=True)      
new_table.to_excel('error_report_%s.xlsx' % today)

excel_formating.format_excel()

In [ ]:
mail_table = new_table.drop(columns=new_table.columns[2:])
mail_table = mail_table['Coverage Advisor'].to_frame().drop_duplicates().reset_index(drop=True)
mail_table = mail_table.merge(mail, how='left', left_on='Coverage Advisor', right_on='Coverage Advisor')
mail_table.dropna(axis=0, subset=['Email'], inplace=True)

### Send Mail

In [ ]:
#outlook server connection
outlServer = smtplib.SMTP(host='smtp-mail.outlook.com', port=587)

#Initialize encryption
outlServer.starttls()

#Login in the outlook account using the define variables username and password
outlServer.login(myMail, password)

In [ ]:
#Definition of the function to send emails
def send_msg(To, From, Subject, Message, s_attach):
    msg = MIMEMultipart()
    msg['From'] = From
    msg['To'] = To
    msg['Subject'] = Subject
    msg_content = MIMEText(Message, 'html', 'utf-8')
    msg.attach(msg_content)
       
    if s_attach == True:
        with open('error_report_%s.xlsx' % today, 'rb') as e:
            exc = MIMEBase('application', 'octet-stream', filename='error_report_%s.xlsx' % today)
            exc.add_header('content-Disposition', 'attachment', filename='error_report_%s.xlsx' % today)
            exc.set_payload(e.read())
            encoders.encode_base64(exc)
            msg.attach(exc)
    
    outlServer.sendmail(From, To, msg.as_string())

In [ ]:
for i, sheet in mail_table.iterrows():
    if pd.isnull(sheet['Email']) == False:
        send_msg(sheet['Email'], myMail, subjectEmail , msg1, True)
        print(sheet['Email'])

In [ ]:
outlServer.quit()